In [1]:
import datetime
import pytz
from semantic_router.llms.openrouter import OpenRouterLLM
from semantic_router import Route, RouteLayer
from semantic_router.encoders import HuggingFaceEncoder
from semantic_router.utils.function_call import get_schema
import geonamescache

class Skill:
    def __init__(self):
        self.geocoder = geonamescache.GeonamesCache()
        self.location = self.geocode_location()
        self.route = Route(
            name='time',
            utterances=[
                "tell me what is the time",
                "what is the date ",
                "time in varshava",
                "date",
                "what date is it today",
                "time in ny",
                "what is the time and date in boston",
                "time",
                "what is the time in makhachkala",
                "date time in st petersburg",
                "what's the date in vienna",
                "date time"
            ],
            function_schema=get_schema(self.run),

        )

        self.rl = RouteLayer(
            encoder=HuggingFaceEncoder(),
            routes=[self.route],
            llm=OpenRouterLLM(
                name='mistralai/mistral-7b-instruct:free',
                openrouter_api_key='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'

            )
        )

    def geocode_location(self, location_name=None):
        if location_name:
            location_name = location_name.title()
            location = list(
                self.geocoder.get_cities_by_name(location_name)[0].values() if self.geocoder.get_cities_by_name(
                    location_name) else self.geocoder.get_us_states_by_names(location_name)[
                    0].values() if self.geocoder.get_us_states_by_names(location_name) else
                self.geocoder.get_countries_by_names(location_name)[
                    0].values() if self.geocoder.get_countries_by_names(location_name) else None)[0]
            return location['timezone']
        else:
            return ''

    def run(self, location:str=None, day:int=0, hour:int=0, minute:int=0):
        """Finds the current time in a specific location.

        :param location: The location to find the current time in, should
            be a valid location. Put the place name itself
            like "rome", or "new york" in the lowercase.
        :type location: str

        :param day: The offset in days from the current date.
            Use positive integers for future dates (e.g., day=1 for tomorrow),
            negative integers for past dates (e.g., day=-1 for yesterday),
            and 0 for the current date.
        :type day: int

        :param hour: The offset in hours from the current time.
            Use positive integers for future times (e.g., hour=1 for one hour ahead),
            negative integers for past times (e.g., hour=-1 for one hour ago),
            and 0 to maintain the current hour.
        :type hour: int

        :param minute: The offset in minutes from the current time.
            Use positive integers for future minutes (e.g., minute=20 for twenty minutes ahead),
            negative integers for past minutes (e.g., minute=-20 for twenty minutes ago),
            and 0 to maintain the current minute.
        :type minute: int

        :return: The time in the specified location."""
        timezone = self.geocode_location(location)
        if timezone:
            tz = pytz.timezone(timezone)
        else:
            tz = None

        current_time = datetime.datetime.now(tz) + datetime.timedelta(days=day)

        # Adding hours and minutes to the current time
        current_time += datetime.timedelta(hours=hour, minutes=minute)

        # Format the date and time as required
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M")

        return formatted_time

s = Skill()
out = s.rl('time in berlin')
print(s.run(**out.function_call))

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-13 17:10:58 INFO semantic_router.utils.logger local
2024-05-13 17:10:59 INFO semantic_router.utils.logger Extracting function input...
2024-05-13 17:11:01 INFO semantic_router.utils.logger LLM output: {
 "location": "berlin"
}
2024-05-13 17:11:01 INFO semantic_router.utils.logger Function inputs: {'location': 'berlin'}


2024-05-13 15:11
